- Uid: u1428543
- Date: May 30, 2023
- Class: CS6017

# Homework 2

## Part 1: A/B Hypothesis Testing

In [ ]:
import scipy.stats as stats

clicks_A = 500
views_A = 1000
clicks_B = 550
views_B = 1000


def get_p_value_two_proportion_z_value_test(clicks_A, views_A, clicks_B, views_B):
    conversion_rate_A = clicks_A / views_A
    conversion_rate_B = clicks_B / views_B

    pooled_conversion_rate = (clicks_A + clicks_B) / (views_A + views_B)

    standard_error = (pooled_conversion_rate * (1 - pooled_conversion_rate) * (1/views_A + 1/views_B)) ** 0.5

    z_value = (conversion_rate_B - conversion_rate_A) / standard_error

    p_value = 1 - stats.norm.cdf(z_value)

    return p_value


p_value = get_p_value_two_proportion_z_value_test(
    clicks_A, views_A, clicks_B, views_B)

print(f"P-value: {p_value:.4f}")

significance_level_1 = 0.05
significance_level_2 = 0.01

null_hypothesis = "The proportion of ad clicks for logo A is equal to the proportion of ad clicks for logo B."

print(f"\nNull Hypothesis: {null_hypothesis}")

def print_conclusion(p_value, significance_level):
    if p_value < significance_level:
        print(f"\nAt the {significance_level*100}% significance level, we reject the null hypothesis. This means that we have enough evidence to conclude that there is a significant difference between the conversion rates of the two logos. The conversion rate of logo B is higher than that of logo A. The observed difference in the data is unlikely to occur by chance alone, assuming the null hypothesis is true.")
    else:
        print(f"\nAt the {significance_level*100}% significance level, we fail to reject the null hypothesis. This means that we do not have enough evidence to conclude a significant difference between the conversion rates at a higher level of confidence. The observed difference in the data could still occur by chance, given the assumptions of the null hypothesis.")

print_conclusion(p_value, 0.01)

print_conclusion(p_value, 0.05)


In [ ]:
print(f"when b is 560")

p_value = get_p_value_two_proportion_z_value_test(clicks_A, views_A, 560, views_B)

print(f"\nP-value: {p_value:.4f}")

print_conclusion(p_value, 0.01)

print_conclusion(p_value, 0.05)


In [ ]:
print(f"when b is 570")

p_value = get_p_value_two_proportion_z_value_test(
    clicks_A, views_A, 570, views_B)

print(f"\nP-value: {p_value:.4f}")

print_conclusion(p_value, 0.01)

print_conclusion(p_value, 0.05)


If the number of clicks for logo B is increased to 560 or 570 times, we would expect the p-value to decrease compared to the original scenario. A smaller p-value suggests a more significant difference in the conversion rates between logo A and logo B, potentially leading to the rejection of the null hypothesis.


## Part 2: Regression of real estate data

### Task 1: Import the Data

In [ ]:
import pandas as pd

df1 = pd.read_csv('realEstate1.csv')

df2 = pd.read_csv('realEstate2.csv')

df = pd.concat([df1, df2])

### Task 2: Clean the Data

In [ ]:
# Only keep houses with List Price between 200,000 and 1,000,000 dollars.

df = df[(df['LstPrice'] >= 200000) & (df['LstPrice'] <= 1000000)]

relevant_columns = ['Acres', 'Deck', 'GaragCap', 'Latitude', 'Longitude', 'LstPrice', 'Patio',
                    'PkgSpacs', 'PropType', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt']

df_relevant = df[relevant_columns].copy()

df_relevant['TotSqf'] = df_relevant['TotSqf'].str.replace(',', '').astype(int)

df_relevant['Prop_Type_SingleFamily'] = (
    df_relevant['PropType'] == 'Single Family').astype(int)

df_relevant = df_relevant[df_relevant['Longitude'] != 0].copy()

df_relevant = df_relevant[(df_relevant['Taxes'] >= 10)
                          & (df_relevant['Taxes'] <= 10000)]


### Task 3: Exploratory Data Analysis



In [ ]:

num_items = len(df_relevant)
print("Number of houses:", num_items)

num_variables = df_relevant.shape[1]
print("Number of variables:", num_variables)

print(df_relevant.describe())


In [ ]:
import matplotlib.pyplot as plt

plt.style.use('ggplot')

house_types = df_relevant['PropType'].value_counts()

print(house_types)

plt.figure(figsize=(10, 6))
house_types.plot(kind='bar')
plt.title('Breakdown of House Types')
plt.xlabel('House Type')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()


In [ ]:
import seaborn as sns
plt.style.use('ggplot')

corr_matrix = df_relevant.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, cmap='RdBu_r', annot=True, fmt='.2f', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()


In [ ]:
plt.style.use('ggplot')

subset_columns = ['Acres', 'LstPrice', 'SoldPrice',
                  'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt']
subset_df = df_relevant[subset_columns]

sns.set(style="ticks")
sns.pairplot(subset_df)
plt.show()


Based on the correlation matrix and the scatter plot matrix, here are some findings:
1. Near perfect correlation:
  The List Price (LstPrice) and Sold Price have a near-perfect positive correlation, as expected. The List Price serves as a starting point for negotiations that lead to the Sold Price, hence the strong correlation.

2. Strong positive correlations:
  Total square footage (TotSqf), number of bathrooms (TotBth), and number of bedrooms (TotBed) exhibit strong positive correlations with the Sold Price. As these property attributes increase, the Sold Price tends to increase as well.

3. Moderate positive correlations:
  Garage capacity (GaragCap) and property size (Acres) show moderate positive correlations with the Sold Price. Properties with larger garage capacities or larger overall sizes tend to have higher Sold Prices.

4. Weak positive correlations:
  Latitude and longitude have weak positive correlations with the Sold Price. While the correlation coefficients between latitude, longitude, and Sold Price may indicate a relatively weak linear relationship, it is important to note that the association between these variables and house prices is not strictly linear but rather multi-linear or spatially related. The observed concentration of higher sold prices on the north and east sides, as well as the lower sold prices on the west and south sides, suggests a discernible spatial pattern in house prices within Salt Lake City. This pattern indicates that the location of a house is a significant factor in determining its price. Therefore, when analyzing the impact of location on house prices, it is essential to consider latitude and longitude collectively in a multi-linear framework, taking into account their spatial relationship rather than relying solely on their individual correlation coefficients.





### Task 4: Geospatial Plot

In [ ]:

plt.style.use('ggplot')

plt.scatter(df_relevant['Longitude'], df_relevant['Latitude'],
            c=df_relevant['SoldPrice'], cmap='coolwarm')
plt.colorbar(label='Sold Price')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('House Sold Price vs. Location')

plt.show()


The observed concentration of higher sold prices on the north and east sides, as well as the lower sold prices on the west and south sides, suggests a discernible spatial pattern in house prices within Salt Lake City. This pattern indicates that the location of a house is a significant factor in determining its price. Factors like the desirability of neighborhoods, proximity to amenities, access to transportation, and other location-specific characteristics likely contribute to these variations in house prices. 

### Bonus: If you can, overlay the scatterplot on a map of the city.

In [ ]:


from branca.colormap import LinearColormap
import folium
from folium.plugins import HeatMap

average_latitude = df_relevant['Latitude'].mean()
average_longitude = df_relevant['Longitude'].mean()

# Create a folium map centered around the average latitude & longitude of the data points
m = folium.Map(location=[average_latitude, average_longitude], zoom_start=14)

colormap = LinearColormap(['blue', 'red'], vmin=df_relevant['SoldPrice'].min(
), vmax=df_relevant['SoldPrice'].max())

for _, row in df_relevant.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius=5,
                        color='transparent',
                        fill_color=colormap(row['SoldPrice']),
                        fill_opacity=0.6).add_to(m)

colormap.caption = 'Sold Price'
colormap.add_to(m)

m


The map is interactive, if it is not rendered on the web, please open the file locally and run the commands.

### Task 5: Simple Linear Regression

In [ ]:

import statsmodels.api as sm

plt.style.use('ggplot')

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
variables = ['Acres', 'LstPrice', 'SoldPrice',
             'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt']

for i, var in enumerate(variables):
    row = i // 3
    col = i % 3

    X = df_relevant[var]
    y = df_relevant['SoldPrice']
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()

    axes[row, col].scatter(
        df_relevant[var], df_relevant['SoldPrice'], label='Actual Data')
    axes[row, col].plot(df_relevant[var], model.fittedvalues,
                        color='blue', label='Regression Line')
    axes[row, col].set_xlabel(var)
    axes[row, col].set_ylabel('Sold Price')
    axes[row, col].set_title(f'{var} vs. Sold Price')
    axes[row, col].legend()

# Format x/y-axis labels ( remove "1e6")
for ax in axes.flat:
    ax.get_yaxis().set_major_formatter(
        plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
    ax.get_xaxis().set_major_formatter(
        plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# Hide empty subplots
if len(variables) < 9:
    for i in range(len(variables), 9):
        row = i // 3
        col = i % 3
        fig.delaxes(axes[row, col])

plt.tight_layout()
plt.show()


As shown in the plots generated above, the variable "List Price" appears to be a strong predictor of the Sold Price. The regression line closely aligns with the actual data points, indicating a strong linear relationship between the List Price and Sold Price. 

In [ ]:
plt.style.use('ggplot')

X = df_relevant['LstPrice']
y = df_relevant['SoldPrice']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

r_squared = model.rsquared
print("R-squared value:", r_squared)

print("\nInterpretation of R-squared:")
print("\nThe R-squared value of", r_squared, "indicates that about",
      r_squared * 100, "% of the variation in the Sold Price can be explained by the List Price.")

plt.scatter(df_relevant['LstPrice'],
            df_relevant['SoldPrice'], label='Actual Data')

plt.plot(df_relevant['LstPrice'], model.fittedvalues,
         color='blue', label='Regression Line')

plt.xlabel('List Price')
plt.ylabel('Sold Price')
plt.title('List Price vs. Sold Price with Regression Line')
plt.legend()
plt.show()


### Task 6: Multilinear Regression

In [ ]:


predictors = ['Acres', 'Taxes', 'TotBed',
              'TotBth', 'TotSqf', 'GaragCap', 'Latitude', 'Longitude']

X = df_relevant[predictors]
y = df_relevant['SoldPrice']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())




1. Which variables are the best predictors for the Sold Price?
   Based on the provided model, the variables Taxes, TotSqf, GaragCap and Longitude appear to be the best predictors for the Sold Price. These variables show both statistical significance (as indicated by low p-values) and meaningful impact (reflected in their coefficient magnitudes) on the SoldPrice.

2. Is the coefficient for TotSqf measuring the price per square foot?
   No, the coefficient for TotSqf does not directly represent the price per square foot. Instead, it represents the estimated change in the Sold Price for each unit increase in the Total Square Footage of a house while holding other variables constant. In this case, for every unit increase in TotSqf, the Sold Price is expected to increase by approximately $45, assuming all other variables in the model remain constant.

3. Estimating the value that each Garage space adds to a house.
   The coefficient for GaragCap indicates that each additional Garage space adds an estimated value of approximately $20,460 to the house, assuming all other variables in the model remain constant.

4. Impact of latitude and longitude on house price.
   - The coefficient for Latitude has a p-value of 0.192, which is greater than 0.05. Therefore, Latitude is not statistically significant at the conventional significance level of 0.05. We cannot conclude with confidence that Latitude has a significant impact on house prices based on this model.
   - On the other hand, the coefficient for Longitude has a p-value of 0.000, which is less than 0.05. This indicates that Longitude is statistically significant, and we can conclude that it has a significant impact on house prices based on this model. The positive coefficient suggests that an increase in Longitude is associated with an increase in the Sold Price.

5. Comparison of model's predictive capability to list price for a house flipping company

In [ ]:

import numpy as np

# List Price Model
X_list = df_relevant['LstPrice']
y_list = df_relevant['SoldPrice']
X_list = sm.add_constant(X_list)
model_list = sm.OLS(y_list, X_list).fit()
y_list_pred = model_list.predict(X_list)

mse_list = np.mean((y_list_pred - y_list) ** 2)
rmse_list = np.sqrt(mse_list)
mae_list = np.mean(np.abs(y_list_pred - y_list))

# Multilinear Regression Model
predictors = ['Acres', 'Taxes', 'TotBed', 'TotBth',
              'TotSqf', 'GaragCap', 'Latitude', 'Longitude']
X_multi = df_relevant[predictors]
y_multi = df_relevant['SoldPrice']
X_multi = sm.add_constant(X_multi)
model_multi = sm.OLS(y_multi, X_multi).fit()
y_multi_pred = model_multi.predict(X_multi)

mse_multi = np.mean((y_multi_pred - y_multi) ** 2)
rmse_multi = np.sqrt(mse_multi)
mae_multi = np.mean(np.abs(y_multi_pred - y_multi))

print("List Price Model:")
print("MSE:", mse_list)
print("RMSE:", rmse_list)
print("MAE:", mae_list)
print("\nMultilinear Regression Model:")
print("MSE:", mse_multi)
print("RMSE:", rmse_multi)
print("MAE:", mae_multi)


The list price model has significantly lower values for MSE, RMSE, and MAE, indicating that it provides better predictions for the sold price compared to the multilinear regression model. Therefore, if you wanted to start a house flipping company, using the list price model would likely result in more accurate price predictions, leading to better decision-making and potentially higher profitability.

## Task 7: Incorporating a Categorical Variable

In [ ]:

plt.style.use('ggplot')

# Model 1: SoldPrice = beta_0 + beta_1 * Prop_Type_SingleFamily
X1 = df_relevant[['Prop_Type_SingleFamily']]
X1 = sm.add_constant(X1)
y = df_relevant['SoldPrice']
model1 = sm.OLS(y, X1).fit()

print("\nModel 1: SoldPrice = beta_0 + beta_1 * Prop_Type_SingleFamily")
print(model1.summary())

In [ ]:
# Model 2: SoldPrice = beta_0 + beta_1 * Prop_Type_SingleFamily + beta_2 * TotSqf
X2 = df_relevant[['Prop_Type_SingleFamily', 'TotSqf']]
X2 = sm.add_constant(X2)
model2 = sm.OLS(y, X2).fit()

print("\nModel 2: SoldPrice = beta_0 + beta_1 * Prop_Type_SingleFamily + beta_2 * TotSqf")
print(model2.summary())


In [ ]:
def plot_scatter(predictions, actual, modelName, ax):
    ax.scatter(predictions, actual)
    ax.plot(actual, actual, color='blue')
    ax.set_xlabel('Predicted Sold Price')
    ax.set_ylabel('Actual Sold Price')
    ax.set_title(modelName)
    ax.ticklabel_format(style='plain')


actual_sold_price = df_relevant['SoldPrice']

predicted_sold_price_model1 = model1.predict(X1)

predicted_sold_price_model2 = model2.predict(X2)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

plot_scatter(predicted_sold_price_model1, actual_sold_price,
             'Model 1:\nSoldPrice = beta_0 + beta_1 * Prop_Type_SingleFamily', axes[0])

plot_scatter(predicted_sold_price_model2, actual_sold_price,
             'Model 2:\nSoldPrice = beta_0 + beta_1 * Prop_Type_SingleFamily + beta_2 * TotSqf', axes[1])

plt.tight_layout()
plt.show()

From the OLS regression results of the first model, we can see that the coefficient for the `Prop_Type_SingleFamily` variable is statistically significant (p = 0 < 0.05), indicating that it has a significant impact on predicting the sold price. The R-squared value for the model is 0.159, suggesting that the `Prop_Type_SingleFamily` variable explains about 15.9% of the variation in the sold price.

On the other hand, the second model includes an additional variable, `TotSqf`, which represents the total square footage of the property. In this model, both the `Prop_Type_SingleFamily` and `TotSqf` variables are included. The coefficient for `Prop_Type_SingleFamily` is no longer statistically significant ( p = 0.415 > 0.05 ), indicating that when we take into account the total square footage, the property type becomes less predictive of the sold price. However, the `TotSqf` variable has a p-value of 0 and a significant coefficient ( 12,650 ), indicating that it has a strong impact on predicting the sold price. The R-squared value for the second model is higher at 0.740, suggesting that the combined effect of `Prop_Type_SingleFamily` and `TotSqf` explains about 74% of the variation in the sold price.

The reason for this change in significance is likely due to confounding between the property type and total square footage. The property type may have a correlation with the total square footage, meaning that the effect of the property type on the sold price is partially explained by the variation in total square footage. When we include the `TotSqf` variable in the model, it captures the variation in sold price associated with the total square footage, thereby reducing the influence of the property type.


In [ ]:
from matplotlib.colors import ListedColormap


property_type = df_relevant['PropType']
total_sqft = df_relevant['TotSqf']
sold_price = df_relevant['SoldPrice']

unique_house_types = property_type.unique()

property_type_mapping = {
    unique_house_types[0]: 1,
    unique_house_types[1]: 2,
    unique_house_types[2]: 3,
}

property_type_numeric = property_type.map(property_type_mapping)

colors = ['red', 'blue', 'green']

fig, ax = plt.subplots(figsize=(10, 6))

scatter = ax.scatter(total_sqft, sold_price,
                     c=property_type_numeric, cmap=ListedColormap(colors))
ax.set_xlabel('Total Sqft')
ax.set_ylabel('Sold Price')

legend_elements = []
for house_type, color in zip(unique_house_types, colors):
    legend_elements.append(plt.Line2D(
        [0], [0], marker='o', color='w', markerfacecolor=color, label=house_type))

ax.legend(handles=legend_elements)

ax.ticklabel_format(style='plain')

plt.title('Scatterplot of TotSqf vs. SoldPrice with Property Type')
plt.show()


From the scatterplot, we can observe the following patterns:

1. Condos: Condos are generally priced lower and have smaller total square footage, typically ranging from 200k to 500k and with total square footage mostly under 2500 square feet.

2. Townhouses: Townhouses fall within a higher price range compared to condos but lower than single-family houses, typically ranging from 300k to 600k. They have a slightly larger total square footage, typically ranging from 1500 to 3000 square feet.

3. Single Family Houses: Single-family houses have a wider range of prices, ranging from 200k to 1000k. They also have the widest range of total square footage, ranging from 1000 to 7000+ square feet. Additionally, we observe a plateau or slight drop in prices after a certain square footage, suggesting diminishing marginal returns.

The total square footage acts as a confounder because it influences both the property type and the sold price. By including total square footage in the model, we account for this confounding variable and find that property type alone is no longer a significant predictor. Total square footage becomes a stronger predictor of sold price because it captures the size and space of the property, which is an essential factor considered by buyers.
